In [103]:
import os;
import json;
import builtins
import pandas as pd;

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge'
testcase = 'ur_uint64_merge'
test_dir = os.path.join(dir, testcase)
results_dir = os.path.join(test_dir, 'outputs', 'results')
csv_dir = os.path.join(test_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)

runs = [os.path.join(results_dir, run) for run in os.listdir(results_dir)]
print(runs)

['/home/chesetti/Repos/learned_merge_cleanup/sponge/ur_uint64_merge/outputs/results/run0', '/home/chesetti/Repos/learned_merge_cleanup/sponge/ur_uint64_merge/outputs/results/run1']


In [104]:
# Walk all the json files and put them in a dataframe

test_results = []
for run in runs:
    for test_result_file in os.listdir(run):
        json_file = builtins.open(os.path.join(run, test_result_file))
        test_result = json.load(json_file)
        test_result['run'] = run
        test_results.append(test_result)
        json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

        

Index(['command', 'run', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.outer_disk_fetch', 'spec.algo', 'spec.algo_name',
       'spec.common_key', 'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'result.inner_index_build_duration_ns',
       'result.inner_index_size', 'spec.index.type', 'spec.threshold',
       'spec.index.leaf_size_in_pages'],
      dtype='object')


# Reports

### Overall Duration

In [105]:
overall_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.duration_ns', aggfunc='median')
overall_duration.to_csv(os.path.join(csv_dir, 'duration_sec.csv'))
print(overall_duration)

spec.algo_name         BTree       PGM_64  standard_merge
spec.common_key                                          
20               223846115.0  170655424.0     159027676.5
40               180772883.0  146829579.5     154406903.0
60               161851763.5  136249658.0     153543930.5
80               150921514.0  128718432.0     152257276.5
100              144420381.0  121792610.5     152282169.5
120              137332904.5  116731518.0     152273980.0
140              131824243.5  113317174.0     152036412.5
160              128806227.0  110951499.0     152673217.5
180              125434111.5  109524113.5     152348108.0
200              123245951.0  106819208.0     151880978.5


In [106]:
inner_index_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_size', aggfunc='median')
inner_index_size.to_csv(os.path.join(csv_dir, 'inner_index_size.csv'))
print(inner_index_size)

spec.algo_name     BTree  PGM_64
spec.common_key                 
20               70032.0  1272.0
40               70032.0  1272.0
60               70032.0  1272.0
80               70032.0  1272.0
100              70032.0  1272.0
120              70032.0  1272.0
140              70032.0  1272.0
160              70032.0  1272.0
180              70032.0  1272.0
200              70032.0  1272.0


In [107]:
inner_index_build_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_build_duration_ns', aggfunc='median')
inner_index_build_duration.to_csv(os.path.join(csv_dir, 'inner_index_build_duration_ns.csv'))

In [108]:
inner_index_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch', aggfunc='median')
print(inner_index_fetch)

spec.algo_name   BTree  PGM_64  standard_merge
spec.common_key                               
20                3907    3907            3907
40                3907    3907            3907
60                3907    3907            3907
80                3907    3907            3907
100               3907    3907            3907
120               3907    3907            3907
140               3907    3907            3907
160               3907    3907            3907
180               3907    3907            3907
200               3907    3907            3907


In [109]:
result_checksum = (test_dataframe[['spec.common_key', 'spec.algo_name', 'result.checksum']].sort_values(by=['spec.common_key', 'spec.algo_name'])) #.loc[test_dataframe['spec.common_key']=='10'])
print(result_checksum.loc[result_checksum['spec.common_key'] == 50])

Empty DataFrame
Columns: [spec.common_key, spec.algo_name, result.checksum]
Index: []
